In [ ]:
# Initial things to use the drone

# from djitellopy import tello
# from time import sleep

# me = tello.Tello()
# To connect 
# me.connect()

# To takeoff
# me.takeoff()

# To control movement
# me.send_rc_control(left/right, forward/backward, up/down, yaw_velocity)

# To stop
# sleep(5)

# To land
# me.land()

Litreature 
- video of drone Face tracking part 
https://www.youtube.com/watch?v=LmEcyQnfpDA

- cv2 github repo to fetch the datasets


In [ ]:
# FACE TRACKING - this works very well

import cv2
import numpy as np

# Detecting a face
def findFace(img):
    faseCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faseCascade.detectMultiScale(imgGray, 1.2, 8)

    myFaceList = []
    myFaceListArea = []

    for(x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cx = x + w//2
        cy = y + h//2
        area = w*h
        cv2.circle(img, (cx, cy), 5, (0, 255, 0), cv2.FILLED)
        myFaceList.append((cx, cy, area))
        myFaceListArea.append(area)
    
    if len(myFaceList)!= 0:
        i = myFaceListArea.index(max(myFaceListArea))
        return img, [myFaceList[i], myFaceListArea[i]]
    
    # Default return to prevent errors
    return img, [[0, 0], 0]
    
# def trackFace(me, info, w, pidf, pError):
#     return


cap = cv2.VideoCapture(0)

while True:
    _, img = cap.read()
    img, info = findFace(img)
    print(f'Area: {info[1]}, Center: {info[0]}')
    cv2.imshow("Face Tracking", img)
    cv2.waitKey(1)

    


In [ ]:
# This is detecting clocks as well as people

import cv2
import numpy as np

# Detecting a person
def findPerson(img):
    personCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_upperbody.xml")
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    persons = personCascade.detectMultiScale(imgGray, 1.1, 4)  # Adjust scaleFactor & minNeighbors if needed

    myPersonList = []
    myPersonListArea = []

    for (x, y, w, h) in persons:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cx = x + w // 2
        cy = y + h // 2
        area = w * h
        cv2.circle(img, (cx, cy), 5, (0, 255, 0), cv2.FILLED)
        myPersonList.append((cx, cy))
        myPersonListArea.append(area)

    if len(myPersonList) != 0:
        i = myPersonListArea.index(max(myPersonListArea))  # Select largest detected person
        return img, [myPersonList[i], myPersonListArea[i]]

    return img, [[0, 0], 0]  # Default return to prevent errors

cap = cv2.VideoCapture(0)

while True:
    _, img = cap.read()
    img, info = findPerson(img)
    print(f'Area: {info[1]}, Center: {info[0]}')

    cv2.imshow("Person Detection", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Press 'q' to exit

cap.release()
cv2.destroyAllWindows()


Area: 7840, Center: (811, 462)
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 21976, Center: (134, 231)
Area: 8466, Center: (808, 463)
Area: 24252, Center: (132, 224)
Area: 21976, Center: (131, 232)
Area: 0, Center: [0, 0]
Area: 24708, Center: (134, 225)
Area: 0, Center: [0, 0]
Area: 30144, Center: (128, 221)
Area: 22110, Center: (132, 226)
Area: 24252, Center: (131, 224)
Area: 0, Center: [0, 0]
Area: 21976, Center: (129, 226)
Area: 0, Center: [0, 0]
Area: 9810, Center: (813, 463)
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 24080, Center: (134, 231)
Area: 21976, Center: (133, 232)
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 20382, Center: (132, 232)
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 25810, Center: (131, 229)
Area: 0, Center: [0

KeyboardInterrupt: 

Using:

- cv2 to capture the image
- Yolo to detect the person

In [ ]:
# This is more accurate

import cv2
from ultralytics import YOLO

# Load YOLOv8 model
# Smallest YOLOv8 model
model = YOLO("yolov8n.pt")  

# Open webcam
cap = cv2.VideoCapture(0) 

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Inference on the frame 
    results = model(frame) 

    myPersonList = []
    myPersonListArea = []

    for r in results:
        for box in r.boxes:
            # Get class ID
            cls = int(box.cls[0]) 
            # Confidence score 
            conf = box.conf[0].item()  
            # Class 0 = "person", confidence > 80%
            if cls == 0 and conf > 0.8:  
                # Get bounding box coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0]) 
                # Center of bounding box
                cx, cy = (x1 + x2) // 2, (y1 + y2) // 2  
                # Calculate area
                area = (x2 - x1) * (y2 - y1)  
                
                # Depending on the size of the bounding box the drone will move
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2) 
                # This is what the drone will follow
                # Draw center point
                cv2.circle(frame, (cx, cy), 5, (0, 255, 0), cv2.FILLED) 

                myPersonList.append((cx, cy))
                myPersonListArea.append(area)

    # Select the largest detected person
    if myPersonList:
        i = myPersonListArea.index(max(myPersonListArea))
        print(f'Area: {myPersonListArea[i]}, Center: {myPersonList[i]}')

    cv2.imshow("Person Detection (YOLOv8)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Press 'q' to exit

cap.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 57.5ms
Speed: 4.5ms preprocess, 57.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Area: 419352, Center: (666, 450)

0: 384x640 1 person, 47.8ms
Speed: 1.0ms preprocess, 47.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Area: 422625, Center: (663, 447)

0: 384x640 1 person, 48.3ms
Speed: 1.4ms preprocess, 48.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Area: 422820, Center: (667, 449)

0: 384x640 1 person, 42.4ms
Speed: 1.5ms preprocess, 42.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Area: 417276, Center: (662, 450)

0: 384x640 1 person, 41.1ms
Speed: 1.2ms preprocess, 41.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Area: 415896, Center: (665, 452)

0: 384x640 1 person, 41.9ms
Speed: 1.4ms preprocess, 41.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Area: 415668, Center: (665, 450)

0: 384x640 1 person, 50.8ms
Speed: 8.5ms prep

KeyboardInterrupt: 